In [2]:
import math;
import numpy as np;
import cv2

In [3]:
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    
    
    imvec = im.reshape(imsz,3);
    
    #Original Old way for DCP 1
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz,1);
    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A1_old = atmsum / numpx;
    
    #New Way with 0.1 percent from DCP 2
    darkvec = dark.ravel()
    count = math.ceil(h * w * 0.1 / 100)
    indices = np.argpartition(darkvec, -count)[:-count]
    A2_new = np.amax(np.take(imvec, indices, axis=0), axis=0)
    
    A2_new = np.reshape(A2_new,A1_old.shape)
    
    print(A2_new) # more
    print(A1_old) # less
    print((A2_new+A1_old)/2)
    print("Next")
    
    return ((A2_new+A1_old)/2)

In [4]:
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

In [5]:
def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

In [6]:
def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;


In [7]:
def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

In [8]:
def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

In [13]:
if __name__ == '__main__':
    import sys
    
    for count in range(1,46):
        
        
        fn = './TrainHazy/'+str(count)+'.png'
        fn2 = './TrainGT/'+str(count)+'.png'
       
        def nothing(*argv):
            pass

        src = cv2.imread(fn);
        src2 = cv2.imread(fn2);
    
        I = src.astype('float64')/255;

        dark = DarkChannel(I,15);
        
        A = AtmLight(I,dark);
        
        #te = TransmissionEstimate(I,A,15);
        #t = TransmissionRefine(src,te);
        #J = Recover(I,t,A,0.1);
        
       
        J = src2.astype('float64')/255
        
        #Save the Temp if you want the RGB Map
        Temp = (I - A)/(J - A)
        
        
        # Decide upon this later. Model will target to get in RGB do Temp above is saved.
        Transmission = np.min(Temp,2) #min, mean or max can be found. Mean was chosen or not ?
        
        
        #Transmission = TransmissionRefine(src2,Transmission);
        
        New = Recover(I,Transmission,A,0.1);
        
        cv2.imwrite("./TransGT/"+str(count)+".png",Temp*255);
        

[[0.81568627 0.73333333 0.71764706]]
[[0.7643076  0.65064134 0.60752655]]
[[0.78999694 0.69198734 0.66258681]]
Next
[[0.73333333 0.71764706 0.71372549]]
[[0.68199755 0.65456087 0.61928513]]
[[0.70766544 0.68610396 0.66650531]]
Next
[[0.63137255 0.59215686 0.59607843]]
[[0.61144608 0.57225082 0.57225082]]
[[0.62140931 0.58220384 0.58416462]]
Next
[[0.88627451 0.79607843 0.7372549 ]]
[[0.85053717 0.72903186 0.63888276]]
[[0.86840584 0.76255515 0.68806883]]
Next
[[0.88627451 0.78431373 0.7254902 ]]
[[0.84269812 0.70943423 0.61928513]]
[[0.86448632 0.74687398 0.67238766]]
Next
[[0.9254902  0.85882353 0.82352941]]
[[0.8544567  0.76822712 0.72511234]]
[[0.88997345 0.81352533 0.77432087]]
Next
[[0.79215686 0.74901961 0.73333333]]
[[0.75646855 0.70943423 0.70159518]]
[[0.7743127  0.72922692 0.71746426]]
Next
[[1. 1. 1.]]
[[0.88189338 0.82310049 0.76038807]]
[[0.94094669 0.91155025 0.88019404]]
Next
[[0.7254902  0.61176471 0.57647059]]
[[0.69767565 0.55657271 0.46642361]]
[[0.71158292 0.5841687